<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/start_swap_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.5 MB/s eta 0:00:00


In [40]:
!pip install -qqq eth_abi

In [18]:
#uni_weth_pool="0x287B0e934ed0439E2a7b1d5F0FC25eA2c24b64f7"
uni_usdc_pool_address="0x6E81F4a9bE879CE9FCBAD4Eafc9311fB9e766299"

In [6]:
from web3 import Web3
from eth_account import Account
import requests
import json
import yfinance as yf
import time

In [26]:
import time, json, decimal
from web3 import exceptions
from eth_abi import encode
#from web3.middleware.proof_of_authority import ExtraDataToPOAMiddleware

In [7]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
address = account.address

print("Connected:", w3.is_connected())
print("Your address:", address)


Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [9]:
#0x3A9D48AB9751398BbFa63ad67599Bb04e4BdF98b

In [11]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  eth_balance = w3.eth.get_balance(address)
  balances.append(["ETH", w3.from_wei(eth_balance, 'ether')])
  return balances

In [14]:
TOKEN_LIST = [
    "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",
    "0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238",
    "0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"
]

In [16]:
check_token_balance(TOKEN_LIST, address)

[['WETH', 0.016],
 ['USDC', 0.0],
 ['UNI', 0.001330846866961009],
 ['ETH', Decimal('0.080329644043361924')]]

In [17]:
usdc_address="0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"
uni_address="0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984"

In [19]:
# --- ABI لازم فقط برای توابع مهم ---
POOL_ABI = [
    {"inputs":[],"name":"slot0","outputs":[
        {"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},
        {"internalType":"int24","name":"tick","type":"int24"},
        {"internalType":"uint16","name":"observationIndex","type":"uint16"},
        {"internalType":"uint16","name":"observationCardinality","type":"uint16"},
        {"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"},
        {"internalType":"uint8","name":"feeProtocol","type":"uint8"},
        {"internalType":"bool","name":"unlocked","type":"bool"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"liquidity","outputs":[
        {"internalType":"uint128","name":"","type":"uint128"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"token0","outputs":[
        {"internalType":"address","name":"","type":"address"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"token1","outputs":[
        {"internalType":"address","name":"","type":"address"}
    ],"stateMutability":"view","type":"function"},

    {"inputs":[],"name":"fee","outputs":[
        {"internalType":"uint24","name":"","type":"uint24"}
    ],"stateMutability":"view","type":"function"},
]

# --- آدرس استخر (Pool Address) را اینجا بگذار ---

pool = w3.eth.contract(address=uni_usdc_pool_address, abi=POOL_ABI)

# --- گرفتن اطلاعات ---
slot0 = pool.functions.slot0().call()
liquidity = pool.functions.liquidity().call()
token0 = pool.functions.token0().call()
token1 = pool.functions.token1().call()
fee = pool.functions.fee().call()

print("slot0 =", slot0)
print("liquidity =", liquidity)
print("token0 =", token0)
print("token1 =", token1)
print("fee =", fee)


slot0 = [154139584199780638519661680902853, 151473, 0, 1, 1, 0, True]
liquidity = 71582917029650
token0 = 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238
token1 = 0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984
fee = 500


In [39]:
import  decimal, sys
# from eth_abi import encode_abi # This line caused the ImportError
# ------------------------- CONFIG -------------------------
WALLET = w3.to_checksum_address(address)  # نمونه
CHAIN_ID = 11155111  # Sepolia

# توکن ها و آدرس پول (مثال‌های شما)
UNI = w3.to_checksum_address(uni_address)
USDC = w3.to_checksum_address(usdc_address)
POOL_ADDRESS = w3.to_checksum_address(uni_usdc_pool_address)

UNIVERSAL_ROUTER = w3.to_checksum_address("0x3A9D48AB9751398BbFa63ad67599Bb04e4BdF98b")

# مقدار سواپ (به واحد توکن اصلی، سپس تبدیل میشود)
AMOUNT_TO_SWAP_HUMAN = 0.0005  # مثال: 0.001 UNI

# ------------------------- ABIs -------------------------
# کوتاه‌شده: ERC20 basics
ERC20_ABI = json.loads("""
[
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"type":"function"},
  {"constant":false,"inputs":[{"name":"spender","type":"address"},{"name":"amount","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"type":"function"}
]
""")

# Uniswap V3 pool ABI (فقط توابع ضروری)
POOL_ABI = json.loads("""
[
  {"inputs":[],"name":"slot0","outputs":[
    {"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},
    {"internalType":"int24","name":"tick","type":"int24"},
    {"internalType":"uint16","name":"observationIndex","type":"uint16"},
    {"internalType":"uint16","name":"observationCardinality","type":"uint16"},
    {"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"},
    {"internalType":"uint8","name":"feeProtocol","type":"uint8"},
    {"internalType":"bool","name":"unlocked","type":"bool"}
  ],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"liquidity","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"}
]
""")

# Universal Router ABI (execute)
UNIVERSAL_ROUTER_ABI = json.loads("""
[
    {"inputs":[{"internalType":"bytes","name":"commands","type":"bytes"},{"internalType":"bytes[]","name":"inputs","type":"bytes[]"}],"name":"execute","outputs":[],"stateMutability":"payable","type":"function"},
    {"inputs":[{"internalType":"bytes","name":"commands","type":"bytes"},{"internalType":"bytes[]","name":"inputs","type":"bytes[]"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"execute","outputs":[],"stateMutability":"payable","type":"function"}
]
""")

# ------------------------- CONTRACTS -------------------------
uni = w3.eth.contract(address=UNI, abi=ERC20_ABI)
usdc = w3.eth.contract(address=USDC, abi=ERC20_ABI)
pool = w3.eth.contract(address=POOL_ADDRESS, abi=POOL_ABI)
universal_router = w3.eth.contract(address=UNIVERSAL_ROUTER, abi=UNIVERSAL_ROUTER_ABI)

# ------------------------- HELPERS -------------------------
def human_to_raw(amount_human, decimals):
    return int(decimal.Decimal(amount_human) * (decimal.Decimal(10) ** decimals))

def raw_to_human(raw, decimals):
    return decimal.Decimal(raw) / (decimal.Decimal(10) ** decimals)

def build_and_send(tx, value=0):
    nonce = w3.eth.get_transaction_count(WALLET)
    gas_price = w3.eth.gas_price
    tx_dict = tx.build_transaction({
        "from": WALLET,
        "nonce": nonce,
        "gasPrice": gas_price,
        "value": value,
        "chainId": CHAIN_ID
    })
    try:
        est = w3.eth.estimate_gas(tx_dict)
        tx_dict['gas'] = int(est * 1.2)
    except Exception as e:
        print("تخمین گس شکست خورد:", e)
        tx_dict.setdefault('gas', 800000)
    signed = w3.eth.account.sign_transaction(tx_dict, PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)
    print("TX sent:", tx_hash.hex())
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=600)
    return receipt

# ------------------------- STEP 0: read pool info -------------------------
def read_pool_info():
    try:
        token0 = pool.functions.token0().call()
        token1 = pool.functions.token1().call()
        fee = pool.functions.fee().call()
        liquidity = pool.functions.liquidity().call()
        slot0 = pool.functions.slot0().call()
        sqrtPriceX96 = slot0[0]
        tick = slot0[1]
        return {
            "token0": token0,
            "token1": token1,
            "fee": fee,
            "liquidity": liquidity,
            "sqrtPriceX96": sqrtPriceX96,
            "tick": tick
        }
    except Exception as e:
        print("خطا در خواندن اطلاعات pool:", e)
        raise

# ------------------------- STEP 1: approve if needed -------------------------
def ensure_approval(token_contract, spender, amount_raw):
    allowance = token_contract.functions.allowance(WALLET, spender).call()
    if allowance >= amount_raw:
        return True
    print("در حال ارسال approve برای", spender, "مقدار:", amount_raw)
    tx = token_contract.functions.approve(spender, 2**256-1)
    receipt = build_and_send(tx)
    return receipt.status == 1

# ------------------------- MAIN: prepare & execute swap -------------------------
def run_swap():
    # اطلاعات Pool
    info = read_pool_info()
    print("Pool info:", info)
    fee = info['fee']
    token0 = info['token0']
    token1 = info['token1']

    # تشخیص جهت (tokenIn=UNI)
    if UNI.lower() == token0.lower():
        token_in = token0
        token_out = token1
        inverted = False
    elif UNI.lower() == token1.lower():
        token_in = token1
        token_out = token0
        inverted = True
    else:
        print("UNI is not part of this pool. Pool tokens:", token0, token1)
        return

    print("token_in:", token_in, "token_out:", token_out, "fee:", fee)

    # decimals
    dec_in = uni.functions.decimals().call()
    dec_out = usdc.functions.decimals().call()

    amount_in_raw = human_to_raw(AMOUNT_TO_SWAP_HUMAN, dec_in)
    print(f"Swapping {AMOUNT_TO_SWAP_HUMAN} (raw {amount_in_raw})")

    # ensure approve
    ok = ensure_approval(uni, UNIVERSAL_ROUTER, amount_in_raw)
    if not ok:
        print("Approve failed")
        return

    # Build Universal Router V3 exactIn command payload
    # The Universal Router V3 exactIn expects: (tokenIn, tokenOut, fee, amountIn, amountOutMinimum, recipient, payWithEth)
    types = ['address','address','uint24','uint256','uint256','address','bool']
    amount_out_min = 1  # conservative; you can compute better estimate from sqrtPrice
    values = [token_in, token_out, fee, amount_in_raw, amount_out_min, WALLET, False]
    encoded = w3.codec.encode(types, values)  # Use w3.codec.encode

    V3_SWAP_EXACT_IN = 0x08
    commands = bytes([V3_SWAP_EXACT_IN])
    inputs = [encoded]

    # try both execute signatures (with deadline and without)
    deadline = int(time.time()) + 120
    try:
        print("Calling universal_router.execute(commands, inputs, deadline)")
        tx = universal_router.functions.execute(commands, inputs, deadline)
        receipt = build_and_send(tx, value=0)
    except Exception as e:
        print("Failed execute with deadline:", e)
        try:
            print("Trying universal_router.execute(commands, inputs) fallback")
            tx = universal_router.functions.execute(commands, inputs)
            receipt = build_and_send(tx, value=0)
        except Exception as e2:
            print("Fallback also failed:", e2)
            return

    print("Receipt status:", receipt.status)
    print("TX hash:", receipt.transactionHash.hex())

    # print balances before/after (approx)
    uni_after_raw = uni.functions.balanceOf(WALLET).call()
    usdc_after_raw = usdc.functions.balanceOf(WALLET).call()
    print("UNI after (raw):", uni_after_raw)
    print("USDC after (raw):", usdc_after_raw)
    print("UNI human:", raw_to_human(uni_after_raw, dec_in))
    print("USDC human:", raw_to_human(usdc_after_raw, dec_out))

# ------------------------- run -------------------------
if __name__ == "__main__":
    global w3 # Declare global to modify the global w3 object
    # Ensure w3 is connected. If not, try to re-establish.
    if not w3.is_connected():
        print("RPC connection lost or not established. Attempting to re-establish...")
        try:
            # Re-initialize w3 to force a new connection attempt
            w3 = Web3(Web3.HTTPProvider(RPC))
            if not w3.is_connected():
                print("Failed to re-establish RPC connection.")
                sys.exit(1) # Still exit if re-establishment fails
            else:
                print("RPC connection re-established.")
        except Exception as e:
            print(f"Error during RPC re-establishment: {e}")
            sys.exit(1)
    print("Connected to RPC:", RPC)
    run_swap()


Connected to RPC: https://eth-sepolia.g.alchemy.com/v2/demo
Pool info: {'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'fee': 500, 'liquidity': 71582917029650, 'sqrtPriceX96': 154139584199780638519661680902853, 'tick': 151473}
token_in: 0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984 token_out: 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238 fee: 500
Swapping 0.0005 (raw 500000000000000)
Calling universal_router.execute(commands, inputs, deadline)
Failed execute with deadline: ('0x3b99b53d', '0x3b99b53d')
Trying universal_router.execute(commands, inputs) fallback
Fallback also failed: ('0x3b99b53d', '0x3b99b53d')


In [47]:
SWAP_ROUTER = "0xE592427A0AEce92De3Edee1F18E0157C05861564"  # Uniswap V3 SwapRouter (common)

# amount to swap (human units)
AMOUNT_IN_HUMAN = decimal.Decimal("0.0005")  # e.g. 0.0005 UNI

# ------------------------- ABIs (minimal) -------------------------
ERC20_ABI = json.loads("""
[
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],"name":"allowance","outputs":[{"name":"uint256","type":"uint256"}],"type":"function"},
  {"constant":false,"inputs":[{"name":"spender","type":"address"},{"name":"amount","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"type":"function"}
]
""")

POOL_ABI = json.loads("""
[
  {"inputs":[],"name":"slot0","outputs":[
    {"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},
    {"internalType":"int24","name":"tick","type":"int24"},
    {"internalType":"uint16","name":"observationIndex","type":"uint16"},
    {"internalType":"uint16","name":"observationCardinality","type":"uint16"},
    {"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"},
    {"internalType":"uint8","name":"feeProtocol","type":"uint8"},
    {"internalType":"bool","name":"unlocked","type":"bool"}
  ],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"liquidity","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
  {"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"}
]
""")

SWAP_ROUTER_ABI = json.loads("""
[
  {
    "inputs": [
      {
        "components": [
          {"internalType":"address","name":"tokenIn","type":"address"},
          {"internalType":"address","name":"tokenOut","type":"address"},
          {"internalType":"uint24","name":"fee","type":"uint24"},
          {"internalType":"address","name":"recipient","type":"address"},
          {"internalType":"uint256","name":"deadline","type":"uint256"},
          {"internalType":"uint256","name":"amountIn","type":"uint256"},
          {"internalType":"uint256","name":"amountOutMinimum","type":"uint256"},
          {"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}
        ],
        "internalType":"struct ISwapRouter.ExactInputSingleParams",
        "name":"params",
        "type":"tuple"
      }
    ],
    "name":"exactInputSingle",
    "outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],
    "stateMutability":"payable",
    "type":"function"
  }
]
""")

# ------------------------- setup web3 & contracts -------------------------
w3 = Web3(Web3.HTTPProvider(RPC))
if not w3.is_connected():
    raise SystemExit("RPC not connected. Check RPC_URL.")

WALLET = w3.to_checksum_address(WALLET)
UNI = w3.to_checksum_address(UNI)
USDC = w3.to_checksum_address(USDC)
POOL_ADDRESS = w3.to_checksum_address(POOL_ADDRESS)
SWAP_ROUTER = w3.to_checksum_address(SWAP_ROUTER)

uni = w3.eth.contract(address=UNI, abi=ERC20_ABI)
usdc = w3.eth.contract(address=USDC, abi=ERC20_ABI)
pool = w3.eth.contract(address=POOL_ADDRESS, abi=POOL_ABI)
router = w3.eth.contract(address=SWAP_ROUTER, abi=SWAP_ROUTER_ABI)

# ------------------------- helpers -------------------------
def human_to_raw(amount_human, decimals):
    return int(decimal.Decimal(amount_human) * (decimal.Decimal(10) ** decimals))

def raw_to_human(raw, decimals):
    return decimal.Decimal(raw) / (decimal.Decimal(10) ** decimals)

def build_send_wait(tx_fn, value=0):
    nonce = w3.eth.get_transaction_count(WALLET)
    gas_price = int(w3.eth.gas_price * 1.0)
    tx = tx_fn.build_transaction({
        "from": WALLET,
        "nonce": nonce,
        "gasPrice": gas_price,
        "value": value,
        "chainId": CHAIN_ID
    })
    # estimate gas safely
    try:
        est = w3.eth.estimate_gas(tx)
        tx['gas'] = int(est * 1.3)
    except Exception:
        tx.setdefault('gas', 800000)
    signed = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
    txh = w3.eth.send_raw_transaction(signed.raw_transaction)
    print("sent tx:", txh.hex())
    receipt = w3.eth.wait_for_transaction_receipt(txh, timeout=600)
    return receipt

# ------------------------- workflow -------------------------
def read_pool():
    s0 = pool.functions.slot0().call()
    liq = pool.functions.liquidity().call()
    t0 = pool.functions.token0().call()
    t1 = pool.functions.token1().call()
    fee = pool.functions.fee().call()
    return {
        "sqrtPriceX96": s0[0],
        "tick": s0[1],
        "liquidity": liq,
        "token0": t0,
        "token1": t1,
        "fee": fee
    }

def ensure_approval(token_contract, spender, amount_raw):
    allowance = token_contract.functions.allowance(WALLET, spender).call()
    if allowance >= amount_raw:
        print("already approved")
        return True
    print("approving token to spender:", spender)
    tx = token_contract.functions.approve(spender, 2**256-1)
    receipt = build_send_wait(tx)
    print("approve receipt:", receipt.status)
    return receipt.status == 1

def swap_exact_input_single(amount_in_human):
    info = read_pool()
    print("pool info:", info)
    fee = info['fee']
    # determine token order
    if UNI.lower() == info['token0'].lower():
        token_in = info['token0']
        token_out = info['token1']
    elif UNI.lower() == info['token1'].lower():
        token_in = info['token1']
        token_out = info['token0']
    else:
        raise SystemExit("UNI not in pool tokens")

    dec_in = uni.functions.decimals().call()
    dec_out = usdc.functions.decimals().call()
    amount_in_raw = human_to_raw(amount_in_human, dec_in)

    # balances before
    uni_before = uni.functions.balanceOf(WALLET).call()
    usdc_before = usdc.functions.balanceOf(WALLET).call()
    eth_before = w3.eth.get_balance(WALLET)

    print("balances before: UNI", raw_to_human(uni_before, dec_in), "USDC", raw_to_human(usdc_before, dec_out), "ETH", eth_before)

    # approve UNI to router
    ok = ensure_approval(uni, SWAP_ROUTER, amount_in_raw)
    if not ok:
        raise SystemExit("approve failed")

    deadline = int(time.time()) + 300
    amount_out_min = 1  # very permissive (you can compute better estimate)
    sqrtPriceLimitX96 = 0

    params = (
        token_in,
        token_out,
        fee,
        WALLET,
        deadline,
        amount_in_raw,
        amount_out_min,
        sqrtPriceLimitX96
    )

    print("calling exactInputSingle with params:", params)
    try:
        receipt = build_send_wait(router.functions.exactInputSingle(params), value=0)
    except Exception as e:
        print("swap call failed:", e)
        raise

    print("swap receipt status:", receipt.status)
    # balances after
    uni_after = uni.functions.balanceOf(WALLET).call()
    usdc_after = usdc.functions.balanceOf(WALLET).call()
    eth_after = w3.eth.get_balance(WALLET)

    print("balances after: UNI", raw_to_human(uni_after, dec_in), "USDC", raw_to_human(usdc_after, dec_out), "ETH", eth_after)

    print("delta UNI:", raw_to_human(uni_after - uni_before, dec_in))
    print("delta USDC:", raw_to_human(usdc_after - usdc_before, dec_out))
    print("ETH gas used (wei):", eth_before - eth_after)

    return receipt

# ------------------------- run -------------------------
if __name__ == "__main__":
    print("connected:", w3.is_connected())
    r = read_pool()
    print("pool read ok:", r)
    swap_exact_input_single(AMOUNT_IN_HUMAN)


connected: False
pool read ok: {'sqrtPriceX96': 154139584199780638519661680902853, 'tick': 151473, 'liquidity': 71582917029650, 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'fee': 500}
pool info: {'sqrtPriceX96': 154139584199780638519661680902853, 'tick': 151473, 'liquidity': 71582917029650, 'token0': '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 'token1': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'fee': 500}
balances before: UNI 0.001330846866961009 USDC 0 ETH 80329490837963512
already approved
calling exactInputSingle with params: ('0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', '0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238', 500, '0x7868EDEfC36A74eba220C75aCFB05c59a4e55822', 1764709314, 500000000000000, 1, 0)
sent tx: 709b1d203f9ef406a77c0399db41ffa81fb3ac86f2fb2392015535b95df06d09
swap receipt status: 1
balances after: UNI 0.001330846866961009 USDC 0 ETH 80329464927963512
delta UNI: 0
delta USDC: 0
ETH gas used (wei

In [48]:
import time
import json
import decimal
from web3 import Web3, exceptions

# --- CONFIGURATION (Assumed global variables) ---
# RPC, PRIVATE_KEY, w3 are assumed to be defined globally and connected.

# --- ADDRESSES ---
# These assume variables 'address', 'uni_address', 'usdc_address', 'uni_usdc_pool_address' are defined elsewhere
WALLET = w3.to_checksum_address(address)
UNI = w3.to_checksum_address(uni_address)
USDC = w3.to_checksum_address(usdc_address)
POOL_ADDRESS = w3.to_checksum_address(uni_usdc_pool_address)
SWAP_ROUTER = w3.to_checksum_address("0xE592427A0AEce92De3Edee1F18E0157C05861564")
CHAIN_ID = 11155111

# --- SWAP PARAMETERS ---
AMOUNT_IN_HUMAN = decimal.Decimal("0.0005")
MAX_UINT256 = 2**256 - 1
MIN_OUT_RAW = 0

# --- ABIs ---
ERC20_ABI = json.loads('''
[{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
{"constant":true,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],"name":"allowance","outputs":[{"name":"uint256","type":"uint256"}],"type":"function"},
{"constant":false,"inputs":[{"name":"spender","type":"address"},{"name":"amount","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"type":"function"}]
''')

POOL_ABI = json.loads('''
[{"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"},
{"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
{"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]
''')

SWAP_ROUTER_ABI = json.loads('''
[{"inputs":[{"components":[{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMinimum","type":"uint256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}],"internalType":"struct ISwapRouter.ExactInputSingleParams","name":"params","type":"tuple"}],"name":"exactInputSingle","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"payable","type":"function"}]
''')

# --- CONTRACT INSTANCES ---
uni = w3.eth.contract(address=UNI, abi=ERC20_ABI)
usdc = w3.eth.contract(address=USDC, abi=ERC20_ABI)
pool = w3.eth.contract(address=POOL_ADDRESS, abi=POOL_ABI)
router = w3.eth.contract(address=SWAP_ROUTER, abi=SWAP_ROUTER_ABI)


# --- HELPERS ---
def human_to_raw(amount_human, decimals):
    return int(decimal.Decimal(amount_human) * (decimal.Decimal(10) ** decimals))

def raw_to_human(raw, decimals):
    return decimal.Decimal(raw) / (decimal.Decimal(10) ** decimals)

def get_token_balance(contract, addr):
    balance_raw = contract.functions.balanceOf(addr).call()
    decimals = contract.functions.decimals().call()
    balance_human = raw_to_human(balance_raw, decimals)
    return balance_raw, decimals, balance_human

def get_eth_balance(addr):
    b = w3.eth.get_balance(addr)
    return b, raw_to_human(b, 18)

def build_send_wait(tx_fn, value=0):
    nonce = w3.eth.get_transaction_count(WALLET)
    gas_price = w3.eth.gas_price
    tx = tx_fn.build_transaction({
        "from": WALLET,
        "nonce": nonce,
        "gasPrice": gas_price,
        "value": value,
        "chainId": CHAIN_ID
    })

    try:
        est = w3.eth.estimate_gas(tx)
        tx['gas'] = int(est * 1.3)
    except Exception:
        tx.setdefault('gas', 800000)

    signed = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
    txh = w3.eth.send_raw_transaction(signed.raw_transaction)
    print("tx hash:", txh.hex())
    receipt = w3.eth.wait_for_transaction_receipt(txh, timeout=600)
    return receipt

# --- WORKFLOW ---
def ensure_approval(token_contract, spender, amount_raw):
    print("\n--- 1. Checking Approval ---")
    allowance = token_contract.functions.allowance(WALLET, spender).call()
    if allowance >= amount_raw:
        print(f"Approval sufficient: {raw_to_human(allowance, token_contract.functions.decimals().call())}")
        return True

    print("Sending Approve transaction...")
    tx = token_contract.functions.approve(spender, MAX_UINT256)
    receipt = build_send_wait(tx)

    if receipt.status == 1:
        print("Approve transaction confirmed.")
        return True
    else:
        print("Approve transaction failed.")
        return False

def swap_exact_input_single():
    print("\n--- 2. Executing Swap (UNI -> USDC) ---")

    # Read Pool Info
    try:
        fee = pool.functions.fee().call()
        token0 = pool.functions.token0().call()
        token1 = pool.functions.token1().call()
    except exceptions.ContractLogicError as e:
        print(f"Error reading pool info. Check pool address/ABI: {e}")
        return

    # Determine token order
    if UNI.lower() == token0.lower():
        token_in = token0
        token_out = token1
    elif UNI.lower() == token1.lower():
        token_in = token1
        token_out = token0
    else:
        raise SystemExit("UNI token address not found in the pool.")

    dec_in = uni.functions.decimals().call()
    dec_out = usdc.functions.decimals().call()
    amount_in_raw = human_to_raw(AMOUNT_IN_HUMAN, dec_in)

    # Balances before
    uni_before_raw, _, uni_before_human = get_token_balance(uni, WALLET)
    usdc_before_raw, _, usdc_before_human = get_token_balance(usdc, WALLET)
    eth_before_raw, eth_before_human = get_eth_balance(WALLET)

    print(f"Balances BEFORE: UNI={uni_before_human:.5f}, USDC={usdc_before_human:.5f}, ETH={eth_before_human:.6f}")

    # Step 1: Approval
    if not ensure_approval(uni, SWAP_ROUTER, amount_in_raw):
        return

    # Step 2: Swap Execution
    deadline = int(time.time()) + 300

    params = (
        token_in,
        token_out,
        fee,
        WALLET, # Recipient is WALLET (your address)
        deadline,
        amount_in_raw,
        MIN_OUT_RAW, # Set to 0
        0 # sqrtPriceLimitX96
    )

    print(f"Calling exactInputSingle with Fee {fee} and amountIn {raw_to_human(amount_in_raw, dec_in)}")
    try:
        tx_fn = router.functions.exactInputSingle(params)
        receipt = build_send_wait(tx_fn, value=0)
    except Exception as e:
        print(f"Swap failed during execution/send: {e}")
        return

    # Check Receipt and Print Diffs
    print(f"Swap receipt status: {receipt.status}")

    uni_after_raw, _, uni_after_human = get_token_balance(uni, WALLET)
    usdc_after_raw, _, usdc_after_human = get_token_balance(usdc, WALLET)
    eth_after_raw, eth_after_human = get_eth_balance(WALLET)

    print("\n--- Balance Changes ---")
    if receipt.status == 1:
        # Check if the swap was successful by looking at token movement
        uni_used = uni_before_raw - uni_after_raw
        usdc_received = usdc_after_raw - usdc_before_raw

        if uni_used > 0 and usdc_received > 0:
            print("Swap SUCCESSFUL. ✅")
        else:
            print("Swap CONFIRMED, but tokens NOT MOVED (Likely low liquidity or pool issue). ⚠️")

        print(f"DELTA UNI (Used): {raw_to_human(uni_used, dec_in):.5f}")
        print(f"DELTA USDC (Received): {raw_to_human(usdc_received, dec_out):.5f}")
    else:
        print("Swap REVERTED. 🛑 Check transaction hash for details.")

    print(f"ETH Gas Used (wei): {eth_before_raw - eth_after_raw}")
    print("-----------------------")

# --- RUN ---
if __name__ == "__main__":
    if w3.is_connected():
        print(f"Connection Status: {w3.is_connected()}")
        swap_exact_input_single()
    else:
        print("Connection Status: False. RPC connection is down.")

Connection Status: True

--- 2. Executing Swap (UNI -> USDC) ---
Balances BEFORE: UNI=0.00133, USDC=0.00000, ETH=0.080329

--- 1. Checking Approval ---
Approval sufficient: 79228162514.26433759354395034
Calling exactInputSingle with Fee 500 and amountIn 0.0005
tx hash: 2d72c9a62f25471c6877049bbe214a34f8f1c915dcad2bc6d99327a94fc10fc1
Swap receipt status: 1

--- Balance Changes ---
Swap CONFIRMED, but tokens NOT MOVED (Likely low liquidity or pool issue). ⚠️
DELTA UNI (Used): 0.00000
DELTA USDC (Received): 0.00000
ETH Gas Used (wei): 25880232920
-----------------------


In [49]:
import time
import json
import decimal
from web3 import Web3, exceptions

# --- CONFIGURATION (Assumed global variables) ---
# RPC, PRIVATE_KEY, w3 are assumed to be defined globally and connected.

# --- ADDRESSES ---
# These assume variables 'address', 'uni_address', 'usdc_address', 'uni_usdc_pool_address' are defined elsewhere
WALLET = w3.to_checksum_address(address)
UNI = w3.to_checksum_address(uni_address)
USDC = w3.to_checksum_address(usdc_address)
POOL_ADDRESS = w3.to_checksum_address(uni_usdc_pool_address)
SWAP_ROUTER = w3.to_checksum_address("0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E")
CHAIN_ID = 11155111

# --- SWAP PARAMETERS ---
AMOUNT_IN_HUMAN = decimal.Decimal("0.0005")
MAX_UINT256 = 2**256 - 1
MIN_OUT_RAW = 0

# --- ABIs ---
ERC20_ABI = json.loads('''
[{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
{"constant":true,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],"name":"allowance","outputs":[{"name":"uint256","type":"uint256"}],"type":"function"},
{"constant":false,"inputs":[{"name":"spender","type":"address"},{"name":"amount","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"type":"function"}]
''')

POOL_ABI = json.loads('''
[{"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"},
{"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
{"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]
''')

SWAP_ROUTER_ABI = json.loads('''
[{"inputs":[{"components":[{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMinimum","type":"uint256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}],"internalType":"struct IV3SwapRouter.ExactInputSingleParams","name":"params","type":"tuple"}],"name":"exactInputSingle","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"payable","type":"function"}]
''')

# --- CONTRACT INSTANCES ---
uni = w3.eth.contract(address=UNI, abi=ERC20_ABI)
usdc = w3.eth.contract(address=USDC, abi=ERC20_ABI)
pool = w3.eth.contract(address=POOL_ADDRESS, abi=POOL_ABI)
router = w3.eth.contract(address=SWAP_ROUTER, abi=SWAP_ROUTER_ABI)


# --- HELPERS ---
def human_to_raw(amount_human, decimals):
    return int(decimal.Decimal(amount_human) * (decimal.Decimal(10) ** decimals))

def raw_to_human(raw, decimals):
    return decimal.Decimal(raw) / (decimal.Decimal(10) ** decimals)

def get_token_balance(contract, addr):
    balance_raw = contract.functions.balanceOf(addr).call()
    decimals = contract.functions.decimals().call()
    balance_human = raw_to_human(balance_raw, decimals)
    return balance_raw, decimals, balance_human

def get_eth_balance(addr):
    b = w3.eth.get_balance(addr)
    return b, raw_to_human(b, 18)

def build_send_wait(tx_fn, value=0):
    nonce = w3.eth.get_transaction_count(WALLET)
    gas_price = w3.eth.gas_price
    tx = tx_fn.build_transaction({
        "from": WALLET,
        "nonce": nonce,
        "gasPrice": gas_price,
        "value": value,
        "chainId": CHAIN_ID
    })

    try:
        est = w3.eth.estimate_gas(tx)
        tx['gas'] = int(est * 1.3)
    except Exception:
        tx.setdefault('gas', 800000)

    signed = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
    txh = w3.eth.send_raw_transaction(signed.raw_transaction)
    print("tx hash:", txh.hex())
    receipt = w3.eth.wait_for_transaction_receipt(txh, timeout=600)
    return receipt

# --- WORKFLOW ---
def ensure_approval(token_contract, spender, amount_raw):
    print("\n--- 1. Checking Approval ---")
    allowance = token_contract.functions.allowance(WALLET, spender).call()
    if allowance >= amount_raw:
        print(f"Approval sufficient: {raw_to_human(allowance, token_contract.functions.decimals().call())}")
        return True

    print("Sending Approve transaction...")
    tx = token_contract.functions.approve(spender, MAX_UINT256)
    receipt = build_send_wait(tx)

    if receipt.status == 1:
        print("Approve transaction confirmed.")
        return True
    else:
        print("Approve transaction failed.")
        return False

def swap_exact_input_single():
    print("\n--- 2. Executing Swap (UNI -> USDC) ---")

    # Read Pool Info
    try:
        fee = pool.functions.fee().call()
        token0 = pool.functions.token0().call()
        token1 = pool.functions.token1().call()
    except exceptions.ContractLogicError as e:
        print(f"Error reading pool info. Check pool address/ABI: {e}")
        return

    # Determine token order
    if UNI.lower() == token0.lower():
        token_in = token0
        token_out = token1
    elif UNI.lower() == token1.lower():
        token_in = token1
        token_out = token0
    else:
        raise SystemExit("UNI token address not found in the pool.")

    dec_in = uni.functions.decimals().call()
    dec_out = usdc.functions.decimals().call()
    amount_in_raw = human_to_raw(AMOUNT_IN_HUMAN, dec_in)

    # Balances before
    uni_before_raw, _, uni_before_human = get_token_balance(uni, WALLET)
    usdc_before_raw, _, usdc_before_human = get_token_balance(usdc, WALLET)
    eth_before_raw, eth_before_human = get_eth_balance(WALLET)

    print(f"Balances BEFORE: UNI={uni_before_human:.5f}, USDC={usdc_before_human:.5f}, ETH={eth_before_human:.6f}")

    # Step 1: Approval
    if not ensure_approval(uni, SWAP_ROUTER, amount_in_raw):
        return

    # Step 2: Swap Execution
    deadline = int(time.time()) + 300

     # ... (قسمت‌های قبلی کد)

    # نکته: در SwapRouter02 ددلاین (deadline) جزو پارامترهای این استراکت نیست
    params = (
        token_in,
        token_out,
        fee,
        WALLET,        # recipient
        amount_in_raw, # amountIn
        MIN_OUT_RAW,   # amountOutMinimum
        0              # sqrtPriceLimitX96
    )

    # ... (ادامه کد)


    print(f"Calling exactInputSingle with Fee {fee} and amountIn {raw_to_human(amount_in_raw, dec_in)}")
    try:
        tx_fn = router.functions.exactInputSingle(params)
        receipt = build_send_wait(tx_fn, value=0)
    except Exception as e:
        print(f"Swap failed during execution/send: {e}")
        return

    # Check Receipt and Print Diffs
    print(f"Swap receipt status: {receipt.status}")

    uni_after_raw, _, uni_after_human = get_token_balance(uni, WALLET)
    usdc_after_raw, _, usdc_after_human = get_token_balance(usdc, WALLET)
    eth_after_raw, eth_after_human = get_eth_balance(WALLET)

    print("\n--- Balance Changes ---")
    if receipt.status == 1:
        # Check if the swap was successful by looking at token movement
        uni_used = uni_before_raw - uni_after_raw
        usdc_received = usdc_after_raw - usdc_before_raw

        if uni_used > 0 and usdc_received > 0:
            print("Swap SUCCESSFUL. ✅")
        else:
            print("Swap CONFIRMED, but tokens NOT MOVED (Likely low liquidity or pool issue). ⚠️")

        print(f"DELTA UNI (Used): {raw_to_human(uni_used, dec_in):.5f}")
        print(f"DELTA USDC (Received): {raw_to_human(usdc_received, dec_out):.5f}")
    else:
        print("Swap REVERTED. 🛑 Check transaction hash for details.")

    print(f"ETH Gas Used (wei): {eth_before_raw - eth_after_raw}")
    print("-----------------------")

# --- RUN ---
if __name__ == "__main__":
    if w3.is_connected():
        print(f"Connection Status: {w3.is_connected()}")
        swap_exact_input_single()
    else:
        print("Connection Status: False. RPC connection is down.")

Connection Status: True

--- 2. Executing Swap (UNI -> USDC) ---
Balances BEFORE: UNI=0.00133, USDC=0.00000, ETH=0.080329

--- 1. Checking Approval ---
Sending Approve transaction...
tx hash: b6ba3606774caa60e225e6461d0461405ede63ec05632c6a6f0991e774d5fbaf
Approve transaction confirmed.
Calling exactInputSingle with Fee 500 and amountIn 0.0005
tx hash: ed865374cfe8e0faf645be56988881d36a231839a49b9ed4baadda68b6ed53d3
Swap receipt status: 1

--- Balance Changes ---
Swap SUCCESSFUL. ✅
DELTA UNI (Used): 0.00050
DELTA USDC (Received): 131.56124
ETH Gas Used (wei): 197847259605
-----------------------
